In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
#!pip install bitsandbytes


In [2]:
from google.colab import drive
drive.mount('/content/drive')

secret_file_path = '/content/drive/My Drive/secrets/huggingfacetoken.key'
with open(secret_file_path) as f:
  hf_token=f.read().strip()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
secret_file_path = '/content/drive/My Drive/secrets/wandb.key'

with open(secret_file_path) as f:
  wb_token=f.read().strip()

In [4]:
from huggingface_hub import login
#user_secrets = UserSecretsClient()

#hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(hf_token)

In [5]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset 2',
    job_type="training",
    anonymous="allow"
)

In [6]:
#!pip install unsloth_zoo
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

In [7]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate standard Finnish sentence to South Ostrobothnian dialect.

### Question:
{}

### Response:
<think>{}"""

In [8]:
question = "Hevoskauppias ajoi uudella hevosellaan meidän talon ohi. Katsoin häntä aitan ovelta."

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to translate the given Finnish sentence into the South Ostrobothnian dialect. Let me start by understanding the original sentence.

The original sentence is: "Hevoskauppias ajoi uudella hevosellaan meidän talon ohi. Katsoin häntä aitan ovelta."

Breaking it down, "Hevoskauppias" is a term, maybe a nickname or a term of endearment. "Ajoi" means he drove or went. "Uudella hevosellaan" is his new horse. "Meidän talon ohi" refers to the outskirts or around our farm. The second sentence: "Katsoin häntä aitan ovelta" means I watched him from the kitchen window.

Now, I need to translate this into the South Ostrobothnian dialect. I know that dialects often have unique terms and phrases. Let me think about the words:

- "Hevoskauppias" might translate to something like "Horseman" or a term of endearment.
- "Ajoi" is "drove" or "went."
- "Uudella hevosellaan" would be "with the new horse."
- "Meidän talon ohi" could be "around our farm."
- "Aitan ovelta" is "from the ki

In [9]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate standard Finnish sentence to South Ostrobothnian dialect.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [10]:
train_prompt_vocabulary_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate the following word from standard Finnish to South Ostrobothnian dialect.

### Word:
{}

### Dialect Equivalent:
<think>
{}
</think>
{}"""

In [11]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [12]:
def formatting_prompts_vocab_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_vocabulary_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [13]:
#import json
from datasets import load_dataset
data_file_path = '/content/drive/My Drive/data/train_set_deep_seek.json'


#with open(data_file_path) as f:
#  dataset = json.load(f)

dataset0 = load_dataset('json',data_files=data_file_path, split = "train[0:350]",trust_remote_code=True)

dataset0 = dataset0.map(formatting_prompts_func, batched = True,)
dataset0["Question"][0]

'Ensimmäisen kerran pääsimme elokuviin seuratalolle kun osuuskaupan kokouksessa näytettiin kuvia eri tehtailta'

In [14]:
data_file_path = '/content/drive/My Drive/data/vocabulary_deep_seek_format.json'


#with open(data_file_path) as f:
#  dataset = json.load(f)

dataset1 = load_dataset('json',data_files=data_file_path, split = "train[:-1]",trust_remote_code=True)

dataset1 = dataset1.map(formatting_prompts_vocab_func, batched = True,)

In [15]:
#from datasets import concatenate_datasets
#dataset=concatenate_datasets([dataset0, dataset1])
#print(dataset)

In [16]:
#from datasets import load_dataset
#dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
print(type(dataset0))
#dataset = dataset.map(formatting_prompts_func, batched = True,)
#print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [17]:
print(dataset0['text'][-10])

Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate standard Finnish sentence to South Ostrobothnian dialect.

### Question:
Uudet paperilakanat eivät kestä kuin muutaman hetken kun Tuomas on niin kova peuhaamaan se oli naapurin emännälle toimittanut

### Response:
<think>

</think>
Uuret paperilakanat ei kestä  ku muutaman siunaaman hetken ku Tuamas on niin kova kiakkaamahan  soli krannin emännälle toimittanu<｜end▁of▁sentence｜>


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer_transl = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset0,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        #max_steps=60,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

In [20]:
#works when not ran
trainer_vocab = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset1,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        #max_steps=60,
        num_train_epochs=1,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs/vocab",
    ),
)

In [21]:
!nvidia-smi
trainer_stats = trainer_transl.train()

Sun Mar 16 19:01:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             31W /   70W |    6180MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 350 | Num Epochs = 3 | Total steps = 129
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,3.302700
20,1.805900
30,1.475600
40,1.438800
50,1.353500
60,1.049900
70,1.081600
80,1.039900
90,1.046200
100,0.776500


In [22]:
!nvidia-smi
trainer_stats = trainer_vocab.train()

Sun Mar 16 19:13:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             32W /   70W |    6928MiB /  15360MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,688 | Num Epochs = 1 | Total steps = 211
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
10,1.257600
20,0.346200
30,0.322800
40,0.322100
50,0.311800
60,0.339900
70,0.302900
80,0.338400
90,0.314400
100,0.289800


In [23]:
question = "Hevoskauppias ajoi uudella hevosellaan meidän talon ohi. Katsoin häntä aitan ovelta."


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>

</think>
Hevooskauppias ajoo uureella hevoosellaan meirän taloon ohi. Kattelin sitä aita oohela<｜end▁of▁sentence｜>


In [24]:
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Number of CUDA Devices: {torch.cuda.device_count()}")
print(f"Current CUDA Device: {torch.cuda.current_device()}")

CUDA Available: True
Number of CUDA Devices: 1
Current CUDA Device: 0


In [25]:
import shutil
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)




Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.41 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 34%|███▍      | 11/32 [00:00<00:01, 14.98it/s]
We will save to Disk and not RAM now.
 78%|███████▊  | 25/32 [00:45<00:12,  1.82s/it]


KeyboardInterrupt: 

In [ ]:
!ls -l

In [ ]:
!ls -l ./DeepSeek-R1-Medical-COT
shutil.make_archive("deepseek_model", 'zip', new_model_local)
